In [1]:
import os
import re
import ast

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.max_columns = None

In [2]:
# read in the data
# the data is split up in multiple .csv-files, so we generate a single dataframe from all of them

# get all filenames in the data directory
os.chdir(os.getcwd()+'/data/data')
all_files = list(filter(os.path.isfile, os.listdir('.')))

# in case there are other files in the directory: 
# define a pattern which all relevant files follow, then make a list of 'valid' files based on their name
regex = re.compile(r"Kickstarter\d+.csv")
valid_files = list(filter(regex.match, all_files))

# read the files and concatenate a single dataframe
df_from_each_file = (pd.read_csv(f) for f in valid_files)
data = pd.concat(df_from_each_file, ignore_index=True)
data.shape

(209222, 37)

First we look at the target variable **state**. There are 5 categories, successful, failed, live, suspended and canceled. Since we want to predict the likelihood of success, we will only consider campaigns that had a regular finish after reaching the deadline (not suspended or canceled). Campaigns that are still live are also of no use to us. So we will drop the corresponding observations.

In [3]:
# drop all rows where state == live/suspended/canceled, these projects are neither successful nor did they fail
data.drop(data[data.state.isin(['live', 'suspended', 'canceled'])].index, inplace=True)
data.shape

(192664, 37)

Interestingly enough, some campaigns seem to be excact duplicates of one another. These rows we will also drop.

In [4]:
data.drop_duplicates(inplace=True, subset='id')
data.shape

(168979, 37)

There are 37 columns in this data set. To make the EDA more comfortable we will now drop some columns, either because they have no meaning/significance for our business case, or they are redundant with other columns. We also need to ensure that the dataframe contains only information that is available at the start of a project (except our target variable **state**). Otherwise, we would be trying to predict whether a project succeeds or fails based on 'future information', like for example the amount of money that was pledged.

We eliminate the following columns:

In [5]:
data.drop(axis=1, inplace=True, labels=[
    'currency_symbol',            # redundant with currency
    'friends',                    # only NaN values
    'fx_rate',                    # 
    'is_backing',                 # only NaN values
    'slug',                       # redundant with name
    'permissions',                # only NaN values
    'photo',                      # we will not use image analysis, so no significance for model
    'profile',                    # contains many values that are also in other columns, too many different profile ids to dummy
    'static_usd_rate',            # data imbalance, usefullness to model unlikely
    'source_url',                 # contains url for categories, thus redundant
    'state_changed_at', 
    'deadline', 
    'id',                         # unique campaign identifier
    'converted_pledged_amount',   # redundant with pledged
    'created_at', 
    'is_starred', 
    'currency',                   # largely redundant with (and less detailed than) country
    'current_currency',
    'disable_communication',      # only 'False' values
    'is_starrable',
    'location',                   # creating a dummy var. for each city not feasable (we also have country for a general location)
    'urls'                        # not used in model           
])

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168979 entries, 1 to 209221
Data columns (total 15 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   backers_count           168979 non-null  int64  
 1   blurb                   168977 non-null  object 
 2   category                168979 non-null  object 
 3   country                 168979 non-null  object 
 4   creator                 168979 non-null  object 
 5   currency_trailing_code  168979 non-null  bool   
 6   goal                    168979 non-null  float64
 7   launched_at             168979 non-null  int64  
 8   name                    168979 non-null  object 
 9   pledged                 168979 non-null  float64
 10  spotlight               168979 non-null  bool   
 11  staff_pick              168979 non-null  bool   
 12  state                   168979 non-null  object 
 13  usd_pledged             168979 non-null  float64
 14  usd_type            

In [6]:
# convert all date variables to datetime format
data.launched_at = pd.to_datetime(data.launched_at, unit='s')

# convert the objects in category into their respective category names 
data.category = data.category.apply(lambda x: ast.literal_eval(x).get('name'))

In [7]:
# create dummies for the remaining categorial variables and merge them with the dataframe
dummies = pd.get_dummies(data[['state', 'country', 'usd_type', 'category']], drop_first=True)
data = pd.concat([data, dummies], axis=1)
data.shape

# the original categorial variables will be dropped later but are still relevant for the EDA
# data.drop(labels = ['country', 'state', 'usd_type', 'category'], axis=1)

(168979, 196)

In [8]:
#fig, ax = plt.subplots(figsize=(30,30))

#sns.heatmap(data.corr().round(2), annot= True,
#            cmap = sns.diverging_palette(230, 20, as_cmap=True), 
#            mask = np.triu(np.ones_like(data.corr(), dtype=bool)))

In [10]:
# Nachrichten an Philipp:
# in der anderen EDA waren delta_created_launched und campaign_length unter den finalen Prädikatoren (vielleicht doch nicht droppen?)
# fx_rate war aus irgendeinem Grund auch drin, dafür country nicht.
# brauchen wir name und pledged?
# einige Kampagnen sind doppelt im Datensatz - sind überraschen viele, habe ich erstmal entfernt!
# data.id.value_counts(dropna=False)
# data[data['id']==1278991287]